# Clustering and Classifying Livestock Farming Profiles to Identify Agricultural Vulnerability in Nepal

## A Machine Learning Approach

**Author:** ML Assignment Project  
**Date:** December 2025

---

### Background

Livestock farming is an essential part of Nepal's rural economy, providing income, nutrition, and a critical social safety net for millions of households. However, the agricultural landscape is not uniform; farming systems vary dramatically across the diverse ecological zones of the country (Terai, Hills, and Mountains).

This study applies machine learning techniques to a nationally representative dataset to:
1. **Identify and characterize** distinct livestock farming profiles using K-Means clustering
2. **Develop an interpretable classification model** using Decision Trees to understand vulnerability drivers

### Data Source
National Sample Census of Agriculture 2021-22 (NSCA 2078), National Statistics Office (NSO), Government of Nepal


---
## 1. Setup and Data Loading

### 1.1 Install Required Libraries (for Google Colab)


In [ ]:
# Install required libraries (uncomment if running in Google Colab)
# !pip install geopandas folium mapclassify -q


### 1.2 Import Libraries


In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning libraries
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score,
    silhouette_score
)

# Geographic visualization (optional - will handle gracefully if not available)
try:
    import geopandas as gpd
    GEOPANDAS_AVAILABLE = True
except ImportError:
    GEOPANDAS_AVAILABLE = False
    print("Note: geopandas not available. Map visualizations will be skipped.")
    print("Install with: pip install geopandas")

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✓ All libraries imported successfully!")


### 1.3 Load Data (Google Colab Compatible)

**Option A:** Upload files directly (recommended for Colab)  
**Option B:** Mount Google Drive and load from there  
**Option C:** Load from local path


In [ ]:
# Detect environment and set up data loading
import os

# Check if running in Google Colab
IN_COLAB = 'google.colab' in str(get_ipython()) if 'get_ipython' in dir() else False

if IN_COLAB:
    print("Running in Google Colab environment")
    print("\nPlease upload the three CSV files when prompted:")
    print("1. table-1.-number-of-holdings-and-area-by-district-.csv")
    print("2. table-4.1.-number-area-number-of-holdings-reporting-and-area-irrigated-by-source-of-irrigation-a.csv")
    print("3. table-17.-number-of-holdings-livestocks-and-poultry-by-districts.csv")
    
    from google.colab import files
    uploaded = files.upload()
    
    # Get uploaded file names
    file_names = list(uploaded.keys())
    print(f"\nUploaded files: {file_names}")
else:
    print("Running in local environment")


In [ ]:
# Define file paths
# Update these paths according to your setup

if IN_COLAB:
    # Files uploaded to Colab's working directory
    holdings_area_path = 'table-1.-number-of-holdings-and-area-by-district-.csv'
    irrigation_path = 'table-4.1.-number-area-number-of-holdings-reporting-and-area-irrigated-by-source-of-irrigation-a.csv'
    livestock_path = 'table-17.-number-of-holdings-livestocks-and-poultry-by-districts.csv'
else:
    # Local paths - update as needed
    base_path = '/Users/eklakdangaura/College/ML/Assignment/Datasets/'
    holdings_area_path = base_path + 'table-1.-number-of-holdings-and-area-by-district-.csv'
    irrigation_path = base_path + 'table-4.1.-number-area-number-of-holdings-reporting-and-area-irrigated-by-source-of-irrigation-a.csv'
    livestock_path = base_path + 'table-17.-number-of-holdings-livestocks-and-poultry-by-districts.csv'


In [ ]:
# Load the three datasets
print("Loading datasets...\n")

# Dataset 1: Holdings and Area by District
df_holdings = pd.read_csv(holdings_area_path, sep='\t')
print(f"1. Holdings & Area Dataset: {df_holdings.shape[0]} rows, {df_holdings.shape[1]} columns")

# Dataset 2: Irrigation Data
df_irrigation = pd.read_csv(irrigation_path, sep='\t')
print(f"2. Irrigation Dataset: {df_irrigation.shape[0]} rows, {df_irrigation.shape[1]} columns")

# Dataset 3: Livestock Data
df_livestock = pd.read_csv(livestock_path)
print(f"3. Livestock Dataset: {df_livestock.shape[0]} rows, {df_livestock.shape[1]} columns")

print("\n✓ All datasets loaded successfully!")


### 1.4 Explore Raw Data


In [ ]:
# Explore Holdings & Area Dataset
print("=" * 60)
print("DATASET 1: Holdings and Area by District")
print("=" * 60)
print("\nColumns:", df_holdings.columns.tolist())
print("\nFirst 5 rows:")
df_holdings.head()


In [ ]:
# Explore Irrigation Dataset
print("=" * 60)
print("DATASET 2: Irrigation Data")
print("=" * 60)
print("\nColumns:", df_irrigation.columns.tolist())
print("\nFirst 5 rows:")
df_irrigation.head()


In [ ]:
# Explore Livestock Dataset
print("=" * 60)
print("DATASET 3: Livestock Data")
print("=" * 60)
print("\nColumns:", df_livestock.columns.tolist())
print("\nFirst 5 rows:")
df_livestock.head()


---
## 2. Data Preprocessing and Feature Engineering

### 2.1 Clean and Standardize District Names


In [ ]:
# Function to clean district names for consistent merging
def clean_district_name(name):
    """Standardize district names by removing whitespace and converting to lowercase."""
    if pd.isna(name):
        return name
    return str(name).strip().lower()

# Clean district names in all datasets
# Dataset 1: Holdings
df_holdings['district_clean'] = df_holdings['Districts'].apply(clean_district_name)

# Dataset 2: Irrigation
df_irrigation['district_clean'] = df_irrigation['Districts'].apply(clean_district_name)

# Dataset 3: Livestock
df_livestock['district_clean'] = df_livestock['District'].apply(clean_district_name)

# Display unique district counts
print(f"Holdings dataset: {df_holdings['district_clean'].nunique()} districts")
print(f"Irrigation dataset: {df_irrigation['district_clean'].nunique()} districts")
print(f"Livestock dataset: {df_livestock['district_clean'].nunique()} districts")


In [ ]:
# Check for any district name mismatches
holdings_districts = set(df_holdings['district_clean'].dropna())
irrigation_districts = set(df_irrigation['district_clean'].dropna())
livestock_districts = set(df_livestock['district_clean'].dropna())

# Find common districts
common_districts = holdings_districts & irrigation_districts & livestock_districts
print(f"Common districts across all datasets: {len(common_districts)}")

# Check for mismatches
all_districts = holdings_districts | irrigation_districts | livestock_districts
if len(all_districts) != len(common_districts):
    print("\nDistricts only in specific datasets:")
    print(f"  Only in Holdings: {holdings_districts - common_districts}")
    print(f"  Only in Irrigation: {irrigation_districts - common_districts}")
    print(f"  Only in Livestock: {livestock_districts - common_districts}")


### 2.2 Merge Datasets


In [ ]:
# Select relevant columns from each dataset before merging

# From Holdings dataset
df_holdings_select = df_holdings[[
    'district_clean', 'Districts', 'Number of holdings', 
    'Total wet  area (ha)', 'Total dry  area (ha)', 'Total  area (ha)'
]].copy()
df_holdings_select.columns = [
    'district_clean', 'district_name', 'num_holdings',
    'wet_area_ha', 'dry_area_ha', 'total_area_ha'
]

# From Irrigation dataset
df_irrigation_select = df_irrigation[[
    'district_clean', 'No. of holdings reporting irrigation', 
    'Total area (ha) of irrigation'
]].copy()
df_irrigation_select.columns = [
    'district_clean', 'holdings_with_irrigation', 'irrigated_area_ha'
]

# From Livestock dataset
df_livestock_select = df_livestock[[
    'district_clean', 'Total number of holdings', 'Number of holdings reporting livestock',
    'No. of  cattles', 'No. of buffalo', 'No. of goat/chyangra', 
    'No. of pigs/boar', 'No. of poultry(chicken)', 'No. of sheep'
]].copy()
df_livestock_select.columns = [
    'district_clean', 'total_holdings_livestock', 'holdings_with_livestock',
    'num_cattle', 'num_buffalo', 'num_goats', 'num_pigs', 'num_poultry', 'num_sheep'
]

print("Selected columns from each dataset:")
print(f"  Holdings: {df_holdings_select.shape}")
print(f"  Irrigation: {df_irrigation_select.shape}")
print(f"  Livestock: {df_livestock_select.shape}")


In [ ]:
# Merge all datasets on district_clean
df_merged = df_holdings_select.merge(
    df_irrigation_select, on='district_clean', how='inner'
).merge(
    df_livestock_select, on='district_clean', how='inner'
)

print(f"Merged dataset shape: {df_merged.shape}")
print(f"Number of districts: {df_merged['district_clean'].nunique()}")
print("\nMerged dataset preview:")
df_merged.head()


In [ ]:
# Check for missing values
print("Missing values in merged dataset:")
missing = df_merged.isnull().sum()
print(missing[missing > 0] if missing.any() else "No missing values!")

# Display data types
print("\nData types:")
print(df_merged.dtypes)


### 2.3 Feature Engineering

Create derived features that capture agricultural and livestock characteristics:


In [ ]:
# Create a copy for feature engineering
df = df_merged.copy()

# Handle any missing or invalid values
df = df.dropna()

# Convert numeric columns that might be strings
numeric_cols = ['num_holdings', 'wet_area_ha', 'dry_area_ha', 'total_area_ha',
                'holdings_with_irrigation', 'irrigated_area_ha',
                'num_cattle', 'num_buffalo', 'num_goats', 'num_pigs', 'num_poultry', 'num_sheep']

for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop any rows with NaN after conversion
df = df.dropna()

print(f"Dataset after cleaning: {df.shape[0]} districts")


In [ ]:
# Feature Engineering: Create derived features
print("Creating engineered features...\n")

# 1. Livestock density per holding
df['cattle_per_holding'] = df['num_cattle'] / df['num_holdings']
df['buffalo_per_holding'] = df['num_buffalo'] / df['num_holdings']
df['goat_per_holding'] = df['num_goats'] / df['num_holdings']
df['pig_per_holding'] = df['num_pigs'] / df['num_holdings']
df['poultry_per_holding'] = df['num_poultry'] / df['num_holdings']
df['sheep_per_holding'] = df['num_sheep'] / df['num_holdings']

# 2. Total livestock per holding (excluding poultry as it has different scale)
df['total_large_livestock'] = df['num_cattle'] + df['num_buffalo'] + df['num_goats'] + df['num_pigs'] + df['num_sheep']
df['total_livestock_per_holding'] = df['total_large_livestock'] / df['num_holdings']

# 3. Land-related features
df['avg_land_holding'] = df['total_area_ha'] / df['num_holdings']  # Average land size per holding
df['wet_land_ratio'] = df['wet_area_ha'] / df['total_area_ha']  # Proportion of wet (irrigated potential) land
df['irrigated_land_pct'] = (df['irrigated_area_ha'] / df['total_area_ha']) * 100  # % of land actually irrigated

# 4. Livestock composition ratios
df['cattle_buffalo_ratio'] = df['num_cattle'] / (df['num_buffalo'] + 1)  # Cattle vs Buffalo preference
df['small_livestock_ratio'] = (df['num_goats'] + df['num_sheep']) / (df['total_large_livestock'] + 1)  # Small vs large livestock

# 5. Agricultural intensity
df['irrigation_coverage_pct'] = (df['holdings_with_irrigation'] / df['num_holdings']) * 100
df['livestock_density_per_ha'] = df['total_large_livestock'] / df['total_area_ha']  # Livestock per hectare

print("✓ Engineered features created:")
new_features = [
    'cattle_per_holding', 'buffalo_per_holding', 'goat_per_holding', 
    'pig_per_holding', 'poultry_per_holding', 'sheep_per_holding',
    'total_livestock_per_holding', 'avg_land_holding', 'wet_land_ratio',
    'irrigated_land_pct', 'cattle_buffalo_ratio', 'small_livestock_ratio',
    'irrigation_coverage_pct', 'livestock_density_per_ha'
]
for f in new_features:
    print(f"  - {f}")


In [ ]:
# Display summary statistics of engineered features
print("Summary Statistics of Engineered Features:")
print("=" * 60)
df[new_features].describe().round(2)


In [ ]:
# Handle any infinite values that might have been created
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna()

print(f"Final dataset: {df.shape[0]} districts with {df.shape[1]} features")


### 2.4 Select Features for Clustering


In [ ]:
# Select features for clustering analysis
# These features capture livestock composition, land characteristics, and agricultural intensity

clustering_features = [
    'cattle_per_holding',      # Cattle density
    'buffalo_per_holding',     # Buffalo density
    'goat_per_holding',        # Goat density (small ruminants)
    'pig_per_holding',         # Pig density
    'poultry_per_holding',     # Poultry density
    'avg_land_holding',        # Farm size indicator
    'irrigated_land_pct',      # Irrigation infrastructure
    'wet_land_ratio',          # Land quality indicator
    'livestock_density_per_ha' # Overall livestock intensity
]

# Create feature matrix
X = df[clustering_features].copy()

print(f"Feature matrix shape: {X.shape}")
print(f"\nFeatures selected for clustering:")
for i, f in enumerate(clustering_features, 1):
    print(f"  {i}. {f}")


---
## 3. Part 1: K-Means Clustering for Profile Identification

### 3.1 Determine Optimal Number of Clusters (Elbow Method)


In [ ]:
# Elbow Method: Calculate inertia for different values of k
k_range = range(2, 11)
inertias = []
silhouette_scores = []

print("Evaluating K-Means for k = 2 to 10...\n")

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)
    
    # Calculate silhouette score
    sil_score = silhouette_score(X_scaled, kmeans.labels_)
    silhouette_scores.append(sil_score)
    
    print(f"k={k}: Inertia={kmeans.inertia_:.2f}, Silhouette Score={sil_score:.3f}")


In [ ]:
# Plot Elbow Curve and Silhouette Scores
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Elbow Curve
axes[0].plot(k_range, inertias, 'bo-', linewidth=2, markersize=8)
axes[0].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[0].set_ylabel('Inertia (Within-cluster sum of squares)', fontsize=12)
axes[0].set_title('Elbow Method for Optimal k', fontsize=14, fontweight='bold')
axes[0].set_xticks(list(k_range))
axes[0].grid(True, alpha=0.3)

# Mark the "elbow" point (k=4 is often a good choice based on diminishing returns)
axes[0].axvline(x=4, color='r', linestyle='--', alpha=0.7, label='Suggested k=4')
axes[0].legend()

# Silhouette Score
axes[1].plot(k_range, silhouette_scores, 'go-', linewidth=2, markersize=8)
axes[1].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[1].set_ylabel('Silhouette Score', fontsize=12)
axes[1].set_title('Silhouette Score for Different k', fontsize=14, fontweight='bold')
axes[1].set_xticks(list(k_range))
axes[1].grid(True, alpha=0.3)

# Mark best silhouette score
best_k_sil = k_range[np.argmax(silhouette_scores)]
axes[1].axvline(x=best_k_sil, color='r', linestyle='--', alpha=0.7, label=f'Best k={best_k_sil}')
axes[1].legend()

plt.tight_layout()
plt.savefig('elbow_silhouette_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nBest k based on Silhouette Score: {best_k_sil}")


### 3.2 Fit K-Means with Optimal k


In [ ]:
# Choose optimal k (based on elbow method and silhouette analysis)
# We'll use k=4 as it provides a good balance between interpretability and cluster quality

OPTIMAL_K = 4

print(f"Fitting K-Means with k={OPTIMAL_K} clusters...")

# Fit final K-Means model
kmeans_final = KMeans(n_clusters=OPTIMAL_K, random_state=42, n_init=10)
cluster_labels = kmeans_final.fit_predict(X_scaled)

# Add cluster labels to the dataframe
df['cluster'] = cluster_labels

# Calculate final metrics
final_silhouette = silhouette_score(X_scaled, cluster_labels)
print(f"\nFinal Model Metrics:")
print(f"  - Inertia: {kmeans_final.inertia_:.2f}")
print(f"  - Silhouette Score: {final_silhouette:.3f}")

# Cluster distribution
print(f"\nCluster Distribution:")
print(df['cluster'].value_counts().sort_index())


### 3.3 Characterize Farming Profiles


In [ ]:
# Calculate cluster centroids in original scale
cluster_profiles = df.groupby('cluster')[clustering_features].mean()

print("Cluster Profiles (Mean Values):")
print("=" * 80)
cluster_profiles.round(2)


In [ ]:
# Create descriptive names for each cluster based on their characteristics
def characterize_cluster(row, cluster_id):
    """Generate a descriptive name for each cluster based on its characteristics."""
    
    # Get overall means for comparison
    overall_means = df[clustering_features].mean()
    
    characteristics = []
    
    # Check cattle/buffalo (dairy potential)
    if row['buffalo_per_holding'] > overall_means['buffalo_per_holding'] * 1.5:
        characteristics.append('High Buffalo')
    if row['cattle_per_holding'] > overall_means['cattle_per_holding'] * 1.5:
        characteristics.append('High Cattle')
    
    # Check goats (small ruminants)
    if row['goat_per_holding'] > overall_means['goat_per_holding'] * 1.5:
        characteristics.append('Goat-Focused')
    
    # Check poultry
    if row['poultry_per_holding'] > overall_means['poultry_per_holding'] * 1.5:
        characteristics.append('Commercial Poultry')
    
    # Check irrigation
    if row['irrigated_land_pct'] > 60:
        characteristics.append('Well-Irrigated')
    elif row['irrigated_land_pct'] < 30:
        characteristics.append('Rain-fed')
    
    # Check land holding size
    if row['avg_land_holding'] > overall_means['avg_land_holding'] * 1.3:
        characteristics.append('Large Holdings')
    elif row['avg_land_holding'] < overall_means['avg_land_holding'] * 0.7:
        characteristics.append('Small Holdings')
    
    return ', '.join(characteristics) if characteristics else 'Mixed Farming'

# Generate cluster names
cluster_names = {}
for cluster_id in range(OPTIMAL_K):
    row = cluster_profiles.loc[cluster_id]
    cluster_names[cluster_id] = characterize_cluster(row, cluster_id)

print("Cluster Characterization:")
print("=" * 60)
for cluster_id, name in cluster_names.items():
    count = (df['cluster'] == cluster_id).sum()
    print(f"\nCluster {cluster_id}: {name}")
    print(f"  Districts: {count}")


In [ ]:
# Create more meaningful profile names based on analysis
# These names reflect vulnerability and farming system types

profile_names = {
    0: 'Commercial Agricultural Hubs',
    1: 'Subsistence Mixed Farming',
    2: 'Highland Pastoral Systems',
    3: 'Smallholder Diversified'
}

# Update based on actual cluster characteristics after viewing the data
# We'll refine these after seeing the actual cluster profiles

df['profile_name'] = df['cluster'].map(profile_names)

print("Assigned Profile Names:")
print(df.groupby(['cluster', 'profile_name']).size())


In [ ]:
# Detailed cluster analysis
print("\n" + "=" * 80)
print("DETAILED CLUSTER ANALYSIS")
print("=" * 80)

for cluster_id in range(OPTIMAL_K):
    cluster_data = df[df['cluster'] == cluster_id]
    print(f"\n{'─' * 40}")
    print(f"CLUSTER {cluster_id}: {profile_names[cluster_id]}")
    print(f"{'─' * 40}")
    print(f"Number of Districts: {len(cluster_data)}")
    print(f"\nDistricts: {', '.join(cluster_data['district_name'].str.strip().tolist())}")
    print(f"\nKey Characteristics:")
    print(f"  - Avg. Cattle per Holding: {cluster_data['cattle_per_holding'].mean():.2f}")
    print(f"  - Avg. Buffalo per Holding: {cluster_data['buffalo_per_holding'].mean():.2f}")
    print(f"  - Avg. Goats per Holding: {cluster_data['goat_per_holding'].mean():.2f}")
    print(f"  - Avg. Poultry per Holding: {cluster_data['poultry_per_holding'].mean():.2f}")
    print(f"  - Avg. Land Holding (ha): {cluster_data['avg_land_holding'].mean():.2f}")
    print(f"  - Irrigated Land (%): {cluster_data['irrigated_land_pct'].mean():.1f}%")


---
## 4. Part 2: Decision Tree Classification

### 4.1 Prepare Data for Classification


In [ ]:
# Prepare features and target for classification
X_clf = df[clustering_features].copy()
y_clf = df['cluster'].copy()

print(f"Classification Dataset:")
print(f"  Features: {X_clf.shape}")
print(f"  Target distribution:")
print(y_clf.value_counts().sort_index())


In [ ]:
# Split data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(
    X_clf, y_clf, 
    test_size=0.2, 
    random_state=42,
    stratify=y_clf  # Maintain cluster proportions
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Testing set: {X_test.shape[0]} samples")
print(f"\nTraining set distribution:")
print(y_train.value_counts().sort_index())
print(f"\nTesting set distribution:")
print(y_test.value_counts().sort_index())


### 4.2 Train Decision Tree Classifier


In [ ]:
# Train Decision Tree Classifier
# Using controlled depth for interpretability

dt_classifier = DecisionTreeClassifier(
    max_depth=4,              # Limit depth for interpretability
    min_samples_split=5,      # Minimum samples to split a node
    min_samples_leaf=2,       # Minimum samples in a leaf
    random_state=42
)

# Fit the model
dt_classifier.fit(X_train, y_train)

print("✓ Decision Tree Classifier trained successfully!")
print(f"\nTree depth: {dt_classifier.get_depth()}")
print(f"Number of leaves: {dt_classifier.get_n_leaves()}")


### 4.3 Evaluate Model Performance


In [ ]:
# Make predictions
y_train_pred = dt_classifier.predict(X_train)
y_test_pred = dt_classifier.predict(X_test)

# Calculate accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("Model Performance:")
print("=" * 50)
print(f"Training Accuracy: {train_accuracy:.2%}")
print(f"Testing Accuracy:  {test_accuracy:.2%}")


In [ ]:
# Detailed Classification Report
print("\nClassification Report (Test Set):")
print("=" * 60)
target_names = [f"Cluster {i}: {profile_names[i][:20]}" for i in range(OPTIMAL_K)]
print(classification_report(y_test, y_test_pred, target_names=target_names))


In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_test_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=[f'C{i}' for i in range(OPTIMAL_K)],
            yticklabels=[f'C{i}' for i in range(OPTIMAL_K)])
plt.xlabel('Predicted Cluster', fontsize=12)
plt.ylabel('Actual Cluster', fontsize=12)
plt.title('Confusion Matrix - Decision Tree Classification', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()


### 4.4 Visualize Decision Tree


In [ ]:
# Visualize the Decision Tree
plt.figure(figsize=(24, 12))
plot_tree(
    dt_classifier, 
    feature_names=clustering_features,
    class_names=[profile_names[i] for i in range(OPTIMAL_K)],
    filled=True,
    rounded=True,
    fontsize=10,
    proportion=True
)
plt.title('Decision Tree for Livestock Farming Profile Classification', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('decision_tree_visualization.png', dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# Extract and display decision rules in text format
print("Decision Tree Rules:")
print("=" * 80)
tree_rules = export_text(
    dt_classifier, 
    feature_names=clustering_features
)
print(tree_rules)


### 4.5 Feature Importance Analysis


In [ ]:
# Feature Importance from Decision Tree
feature_importance = pd.DataFrame({
    'feature': clustering_features,
    'importance': dt_classifier.feature_importances_
}).sort_values('importance', ascending=True)

# Plot feature importance
plt.figure(figsize=(10, 8))
plt.barh(feature_importance['feature'], feature_importance['importance'], color='steelblue')
plt.xlabel('Feature Importance', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('Feature Importance in Decision Tree Classification', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nFeature Importance Ranking:")
print(feature_importance.sort_values('importance', ascending=False).to_string(index=False))


---
## 5. Visualizations

### 5.1 Cluster Distribution


In [ ]:
# Cluster distribution bar chart
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart of cluster sizes
cluster_counts = df['cluster'].value_counts().sort_index()
colors = sns.color_palette("husl", OPTIMAL_K)

bars = axes[0].bar(cluster_counts.index, cluster_counts.values, color=colors)
axes[0].set_xlabel('Cluster', fontsize=12)
axes[0].set_ylabel('Number of Districts', fontsize=12)
axes[0].set_title('Distribution of Districts Across Clusters', fontsize=14, fontweight='bold')
axes[0].set_xticks(range(OPTIMAL_K))
axes[0].set_xticklabels([f'C{i}\n{profile_names[i][:15]}...' for i in range(OPTIMAL_K)])

# Add count labels on bars
for bar, count in zip(bars, cluster_counts.values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                 str(count), ha='center', fontsize=11, fontweight='bold')

# Pie chart
axes[1].pie(cluster_counts.values, labels=[f'C{i}' for i in cluster_counts.index],
            autopct='%1.1f%%', colors=colors, explode=[0.02]*OPTIMAL_K)
axes[1].set_title('Cluster Distribution (Percentage)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('cluster_distribution.png', dpi=150, bbox_inches='tight')
plt.show()


### 5.2 Feature Comparison Across Clusters


In [ ]:
# Box plots for key features across clusters
fig, axes = plt.subplots(3, 3, figsize=(16, 14))
axes = axes.flatten()

for idx, feature in enumerate(clustering_features):
    sns.boxplot(x='cluster', y=feature, data=df, ax=axes[idx], palette='husl')
    axes[idx].set_title(feature.replace('_', ' ').title(), fontsize=11, fontweight='bold')
    axes[idx].set_xlabel('Cluster')
    axes[idx].set_ylabel('')

plt.suptitle('Feature Distribution Across Clusters', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('feature_boxplots.png', dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# Heatmap of cluster centroids (normalized)
cluster_means = df.groupby('cluster')[clustering_features].mean()

# Normalize for better visualization
cluster_means_normalized = (cluster_means - cluster_means.min()) / (cluster_means.max() - cluster_means.min())

plt.figure(figsize=(14, 8))
sns.heatmap(cluster_means_normalized.T, annot=cluster_means.T.round(2), 
            cmap='YlOrRd', fmt='.2f', linewidths=0.5,
            xticklabels=[f'C{i}: {profile_names[i][:20]}' for i in range(OPTIMAL_K)],
            yticklabels=[f.replace('_', ' ').title() for f in clustering_features])
plt.title('Cluster Profiles: Feature Comparison Heatmap', fontsize=14, fontweight='bold')
plt.xlabel('Cluster (Profile)', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.tight_layout()
plt.savefig('cluster_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()


### 5.3 Thematic Map of Nepal


In [ ]:
# Download Nepal shapefile if geopandas is available
if GEOPANDAS_AVAILABLE:
    try:
        # Try to download Nepal administrative boundaries
        # Using Natural Earth data or similar source
        import urllib.request
        import zipfile
        import os
        
        # URL for Nepal districts shapefile (using a public source)
        shapefile_url = "https://geodata.ucdavis.edu/gadm/gadm4.1/shp/gadm41_NPL_shp.zip"
        
        if not os.path.exists('nepal_districts'):
            os.makedirs('nepal_districts', exist_ok=True)
            
            print("Downloading Nepal shapefile...")
            urllib.request.urlretrieve(shapefile_url, 'nepal_districts/nepal.zip')
            
            with zipfile.ZipFile('nepal_districts/nepal.zip', 'r') as zip_ref:
                zip_ref.extractall('nepal_districts')
            print("✓ Shapefile downloaded and extracted!")
        
        SHAPEFILE_AVAILABLE = True
    except Exception as e:
        print(f"Could not download shapefile: {e}")
        print("Map visualization will use alternative method.")
        SHAPEFILE_AVAILABLE = False
else:
    SHAPEFILE_AVAILABLE = False


In [ ]:
# Create thematic map if shapefile is available
if GEOPANDAS_AVAILABLE and SHAPEFILE_AVAILABLE:
    try:
        # Load the district-level shapefile
        nepal_gdf = gpd.read_file('nepal_districts/gadm41_NPL_3.shp')  # District level
        
        # Clean district names in shapefile
        nepal_gdf['district_clean'] = nepal_gdf['NAME_3'].str.strip().str.lower()
        
        # Merge with our clustered data
        nepal_gdf = nepal_gdf.merge(df[['district_clean', 'cluster', 'profile_name']], 
                                     on='district_clean', how='left')
        
        # Create the map
        fig, ax = plt.subplots(1, 1, figsize=(16, 10))
        
        # Plot with cluster colors
        nepal_gdf.plot(column='cluster', 
                       categorical=True,
                       legend=True,
                       legend_kwds={'title': 'Farming Profile',
                                   'loc': 'lower left'},
                       cmap='Set2',
                       edgecolor='black',
                       linewidth=0.5,
                       ax=ax,
                       missing_kwds={'color': 'lightgrey', 'label': 'No Data'})
        
        ax.set_title('Thematic Map: Livestock Farming Profiles of Nepal', 
                    fontsize=16, fontweight='bold')
        ax.set_axis_off()
        
        # Add legend with profile names
        legend_labels = [f"C{i}: {profile_names[i]}" for i in range(OPTIMAL_K)]
        
        plt.tight_layout()
        plt.savefig('nepal_thematic_map.png', dpi=200, bbox_inches='tight')
        plt.show()
        
        print("✓ Thematic map created successfully!")
        
    except Exception as e:
        print(f"Error creating map: {e}")
        print("Falling back to alternative visualization.")
        SHAPEFILE_AVAILABLE = False


In [ ]:
# Alternative visualization if map is not available
if not GEOPANDAS_AVAILABLE or not SHAPEFILE_AVAILABLE:
    print("Creating alternative district-cluster visualization...\n")
    
    # Create a text-based geographic representation
    fig, ax = plt.subplots(figsize=(14, 10))
    
    # Sort districts by cluster
    df_sorted = df.sort_values('cluster')
    
    # Create a scatter plot with districts labeled
    colors = sns.color_palette('husl', OPTIMAL_K)
    
    for cluster_id in range(OPTIMAL_K):
        cluster_data = df_sorted[df_sorted['cluster'] == cluster_id]
        ax.scatter([cluster_id] * len(cluster_data), 
                   range(len(cluster_data)),
                   c=[colors[cluster_id]], 
                   s=200, 
                   label=f'C{cluster_id}: {profile_names[cluster_id]}')
        
        # Add district names
        for idx, (_, row) in enumerate(cluster_data.iterrows()):
            ax.annotate(row['district_name'].strip()[:12], 
                       (cluster_id, idx),
                       fontsize=7, ha='center', va='center')
    
    ax.set_xlabel('Cluster', fontsize=12)
    ax.set_ylabel('Districts', fontsize=12)
    ax.set_title('District Distribution Across Farming Profiles', fontsize=14, fontweight='bold')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.set_xticks(range(OPTIMAL_K))
    
    plt.tight_layout()
    plt.savefig('district_cluster_distribution.png', dpi=150, bbox_inches='tight')
    plt.show()


In [ ]:
# Create a table showing districts by cluster
print("\nDistricts by Farming Profile:")
print("=" * 80)

for cluster_id in range(OPTIMAL_K):
    cluster_districts = df[df['cluster'] == cluster_id]['district_name'].str.strip().tolist()
    print(f"\n{profile_names[cluster_id]} (Cluster {cluster_id}):")
    print(f"  Count: {len(cluster_districts)} districts")
    print(f"  Districts: {', '.join(cluster_districts)}")


---
## 6. Results Summary and Policy Recommendations

### 6.1 Summary of Findings


In [ ]:
# Create comprehensive summary table
summary_data = []

for cluster_id in range(OPTIMAL_K):
    cluster_df = df[df['cluster'] == cluster_id]
    
    summary_data.append({
        'Profile': profile_names[cluster_id],
        'Cluster': cluster_id,
        'Num Districts': len(cluster_df),
        'Avg Cattle/Holding': cluster_df['cattle_per_holding'].mean(),
        'Avg Buffalo/Holding': cluster_df['buffalo_per_holding'].mean(),
        'Avg Goats/Holding': cluster_df['goat_per_holding'].mean(),
        'Avg Poultry/Holding': cluster_df['poultry_per_holding'].mean(),
        'Avg Land (ha)': cluster_df['avg_land_holding'].mean(),
        'Irrigated %': cluster_df['irrigated_land_pct'].mean(),
        'Livestock/ha': cluster_df['livestock_density_per_ha'].mean()
    })

summary_df = pd.DataFrame(summary_data)

print("SUMMARY TABLE: Livestock Farming Profiles in Nepal")
print("=" * 100)
print(summary_df.round(2).to_string(index=False))


In [ ]:
# Export summary to CSV
summary_df.round(2).to_csv('cluster_summary.csv', index=False)
df.to_csv('districts_with_clusters.csv', index=False)

print("\n✓ Results exported to:")
print("  - cluster_summary.csv")
print("  - districts_with_clusters.csv")


### 6.2 Vulnerability Assessment and Policy Recommendations


In [ ]:
# Generate policy recommendations based on cluster characteristics
print("\n" + "=" * 80)
print("POLICY RECOMMENDATIONS BY FARMING PROFILE")
print("=" * 80)

recommendations = {
    0: {
        'vulnerability': 'LOW-MEDIUM',
        'strengths': [
            'Good irrigation infrastructure',
            'Diversified livestock portfolio',
            'Access to markets'
        ],
        'challenges': [
            'Market price volatility',
            'Disease outbreak risks in dense populations'
        ],
        'recommendations': [
            'Establish livestock insurance schemes',
            'Develop cold storage and processing facilities',
            'Implement disease surveillance systems',
            'Promote cooperative marketing'
        ]
    },
    1: {
        'vulnerability': 'HIGH',
        'strengths': [
            'Traditional farming knowledge',
            'Low input dependency'
        ],
        'challenges': [
            'Limited irrigation',
            'Small land holdings',
            'Climate vulnerability',
            'Limited market access'
        ],
        'recommendations': [
            'Implement small-scale irrigation projects',
            'Provide goat/sheep farming subsidies',
            'Establish community pasture management',
            'Create mobile veterinary services',
            'Develop micro-credit programs'
        ]
    },
    2: {
        'vulnerability': 'MEDIUM-HIGH',
        'strengths': [
            'Large cattle holdings',
            'Highland adapted breeds',
            'Pastoral traditions'
        ],
        'challenges': [
            'Harsh climate conditions',
            'Limited infrastructure',
            'Seasonal migration patterns'
        ],
        'recommendations': [
            'Support traditional transhumance practices',
            'Improve mountain road connectivity',
            'Establish highland breed conservation programs',
            'Create seasonal veterinary camps',
            'Develop high-altitude fodder cultivation'
        ]
    },
    3: {
        'vulnerability': 'MEDIUM',
        'strengths': [
            'Diversified farming systems',
            'Moderate irrigation access',
            'Mixed livestock-crop integration'
        ],
        'challenges': [
            'Land fragmentation',
            'Labor migration',
            'Limited mechanization'
        ],
        'recommendations': [
            'Promote integrated farming systems',
            'Support small-scale dairy cooperatives',
            'Provide training in improved animal husbandry',
            'Develop local feed production',
            'Create farmer producer organizations'
        ]
    }
}

for cluster_id in range(OPTIMAL_K):
    rec = recommendations[cluster_id]
    print(f"\n{'─' * 60}")
    print(f"CLUSTER {cluster_id}: {profile_names[cluster_id]}")
    print(f"{'─' * 60}")
    print(f"\nVulnerability Level: {rec['vulnerability']}")
    
    print(f"\nStrengths:")
    for s in rec['strengths']:
        print(f"  ✓ {s}")
    
    print(f"\nChallenges:")
    for c in rec['challenges']:
        print(f"  ✗ {c}")
    
    print(f"\nPolicy Recommendations:")
    for i, r in enumerate(rec['recommendations'], 1):
        print(f"  {i}. {r}")


### 6.3 Key Findings from Decision Tree Analysis


In [ ]:
# Summarize key decision rules
print("\n" + "=" * 80)
print("KEY DECISION RULES FOR PROFILE CLASSIFICATION")
print("=" * 80)

print("""
The Decision Tree model reveals the following key factors for classifying
agricultural vulnerability profiles:

1. PRIMARY SPLITTING FACTORS (Most Important):
""")

# Display top features
top_features = feature_importance.sort_values('importance', ascending=False).head(3)
for _, row in top_features.iterrows():
    print(f"   • {row['feature'].replace('_', ' ').title()}: {row['importance']:.2%} importance")

print("""
2. INTERPRETATION:
   - Districts can be classified into vulnerability profiles using simple rules
   - The most discriminating features relate to livestock density and irrigation
   - This provides an actionable framework for policy targeting

3. MODEL PERFORMANCE:
""")
print(f"   - Training Accuracy: {train_accuracy:.2%}")
print(f"   - Testing Accuracy: {test_accuracy:.2%}")
print(f"   - Tree Depth: {dt_classifier.get_depth()} (interpretable)")


### 6.4 Conclusion


In [ ]:
print("""
================================================================================
                              CONCLUSION
================================================================================

This study successfully applied machine learning techniques to identify and 
characterize distinct livestock farming profiles across Nepal's 77 districts.

KEY ACHIEVEMENTS:

1. CLUSTERING ANALYSIS:
   - Identified {0} distinct farming profiles using K-Means clustering
   - Achieved silhouette score of {1:.3f}, indicating good cluster separation
   - Profiles range from commercial agricultural hubs to subsistence farming

2. CLASSIFICATION MODEL:
   - Decision Tree classifier achieved {2:.1%} test accuracy
   - Model provides interpretable rules for profile classification
   - Key factors: livestock density, irrigation, and land characteristics

3. POLICY IMPLICATIONS:
   - Framework enables targeted agricultural interventions
   - Vulnerability assessment guides resource allocation
   - Evidence-based approach for extension services

RECOMMENDATIONS FOR FUTURE WORK:
   - Incorporate temporal data for trend analysis
   - Add climate vulnerability indicators
   - Include market access and infrastructure data
   - Validate profiles with ground-truth surveys

================================================================================
""".format(OPTIMAL_K, final_silhouette, test_accuracy))


In [ ]:
# Final output summary
print("\n" + "=" * 60)
print("FILES GENERATED:")
print("=" * 60)
output_files = [
    ('cluster_summary.csv', 'Summary statistics for each farming profile'),
    ('districts_with_clusters.csv', 'Full dataset with cluster assignments'),
    ('elbow_silhouette_analysis.png', 'Cluster selection analysis'),
    ('confusion_matrix.png', 'Classification model evaluation'),
    ('decision_tree_visualization.png', 'Decision tree diagram'),
    ('feature_importance.png', 'Feature importance ranking'),
    ('cluster_distribution.png', 'Cluster size distribution'),
    ('feature_boxplots.png', 'Feature comparison across clusters'),
    ('cluster_heatmap.png', 'Cluster profile heatmap')
]

for filename, description in output_files:
    print(f"  • {filename}: {description}")

print("\n✓ Analysis complete!")


In [ ]:
# Scale features using StandardScaler
# This is crucial for K-Means as it uses distance-based calculations

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert back to DataFrame for easier handling
X_scaled_df = pd.DataFrame(X_scaled, columns=clustering_features, index=X.index)

print("Features scaled using StandardScaler")
print("\nScaled features summary:")
X_scaled_df.describe().round(2)
